In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from py_lex import EmoLex
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import nltk
from collections import Counter
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius as lg
import re


Esta notebook contiene todo el trabajo hecho para el armado del dataset del desafío 3 (análisis de Spotify) 
y todo el armado complementario para el desafio final (análisis de Emociones de letra de Canciones: Lexicon + Genius)

# <b> Índice </b>

#### <b><a href="#Spotify">1. Spotify</a></b>
#### <b><a href="#Genius">2. Genius</a></b>
#### <b><a href="#Lexicon">3. Lexicon</a></b>














---


## <b> <a id="Spotify">1. Comenzamos con la utilizacion de Spotipy para descargar todas las features de las Playlist que seleccionamos de Spotify </a> </b>
### Configuración Spotipy (generar client_id y client_secret en Spotify)

In [2]:
sp = spotipy.Spotify() 
cid ="INGRESAR_CLIENT_ID"
secret = "729931684d3249b998dd12f8378f0e40"
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 
sp.trace=False

### Funcion para descargar features de tracks de SPOTIFY

In [11]:
def analyze_playlist(creator, playlist_id, genre):
    
    # Create empty dataframe
    playlist_features_list = ["artist","album","track_name","track_id","danceability","energy","key","loudness","mode","speechiness","instrumentalness","valence","tempo", "time_signature", "acousticness"]

    playlist_df = pd.DataFrame(columns = playlist_features_list)

    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["album_release_date"] = track["track"]["album"]["release_date"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        playlist_features["track_popularity"] = track["track"]["popularity"]/10
        playlist_features["duration_s"] = track["track"]["duration_ms"]        
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    def to_datetime_year(playlist_df):
        playlist_df["album_release_year"] = pd.to_datetime(playlist_df['album_release_date']).dt.year
        playlist_df.drop(['album_release_date'], axis=1, inplace=True)
    to_datetime_year(playlist_df)
    playlist_df["duration_s"] =  (playlist_df["duration_s"]/1000).round()
    playlist_df.insert(0, "genre", genre, True) 
    
    return playlist_df

---

Armamos 5 listas, 1 para cada genero, y buscamos en Spotify (manualmente) Playlist del tipo "This is" de bandas de cada genero. Tomamos el ID de la Playlist.
Con esta información, podemos buscar en la API de Spotify toda la información realicionada a cada Track.
##### <b> NO EJECUTAR CON TODOS EL LISTADO DE TRACKS YA QUE DEMORA MUCHAS HORAS. </b> 

** <b> Para probar la notebook se deja solo un playlist_id </b>

In [15]:
dt_columns = ["genre", "artist","album","track_name","track_id","danceability","energy","key","loudness","mode","speechiness","instrumentalness","valence","tempo", "time_signature", "acousticness"]
spotify_df = pd.DataFrame(columns = dt_columns)

ejecutar = True ## '******************* Ejecucion deshabilitada *******************'
if ejecutar:
    
    list_pop = ["37i9dQZF1DX6bnzK9KPvrz"]
    for x in list_pop:
        print('playlist_id: ' + x)
        new_list = analyze_playlist("spotify", x, "Pop") 
        spotify_df = pd.concat([spotify_df, new_list])
    
    #list_jazz = ("37i9dQZF1DX3BqHI5fuXWV","37i9dQZF1DWVx3vT1QCKCV","37i9dQZF1DZ06evO2PODB2","37i9dQZF1DXciLbPVkluM0","37i9dQZF1DZ06evO3shPxb","37i9dQZF1DZ06evO15Oxjn","37i9dQZF1DZ06evO1NHKFo","37i9dQZF1DZ06evO1jYxGw","37i9dQZF1DZ06evO3J3yBN","37i9dQZF1DZ06evO0wu1S1","37i9dQZF1DZ06evO4rmk92","37i9dQZF1DWTKezOYpetsD","37i9dQZF1DZ06evO2TXsYJ","37i9dQZF1DZ06evO17hgFa","37i9dQZF1DZ06evO15wnBo","37i9dQZF1DZ06evO2pcdar","37i9dQZF1DZ06evO2wKKgo","37i9dQZF1DZ06evNZYUXCF","37i9dQZF1DZ06evO0kFbAA")
    #for x in list_jazz:
    #  new_list = analyze_playlist("spotify", x, "Jazz") 
    #  spotify_df = pd.concat([spotify_df, new_list])

    #list_pop = ("37i9dQZF1DXaTIN6XNquoW","37i9dQZF1DXc7FZ2VBjaeT","37i9dQZF1DX1PfYnYcpw8w","37i9dQZF1DX6bnzK9KPvrz","37i9dQZF1DX2jD2cAwnIit","37i9dQZF1DX7KWRIlkUpsl","37i9dQZF1DXc2aPBXGmXrt","37i9dQZF1DX3XuY09ikg3x","37i9dQZF1DWWxPM4nWdhyI","37i9dQZF1DWSPcmr9PNm5H","37i9dQZF1DWXcA2XXbXQ3d","37i9dQZF1DWVtNGBlJrHWR","37i9dQZF1DX3lQi4RhMuwd","37i9dQZF1DX29brXfjEm5q","37i9dQZF1DWWUJdr9ahsbf","37i9dQZF1DX9DHKgiu5R9j")
    #for x in list_pop:
    #  new_list = analyze_playlist("spotify", x, "Pop") 
    #  spotify_df = pd.concat([spotify_df, new_list])

    #list_heavymetal = ("37i9dQZF1DXc4xFsxShkAv","37i9dQZF1DX9qWLnBVC7yq","37i9dQZF1DX5WtNRbfbuUo","37i9dQZF1DWZI1Z6Eq6Z2C","37i9dQZF1DXbj9Ksq4BAdj","37i9dQZF1DX8MtmyPcHg1L","37i9dQZF1DX3UiIXvG3mSY","37i9dQZF1DX3tY3fLFgLOc","37i9dQZF1DXagb3XZSnOQ8","37i9dQZF1DZ06evO35omLA","37i9dQZF1DZ06evO2HXM8g","37i9dQZF1DXaBkG9rHSMqy","37i9dQZF1DZ06evO44jwGW","37i9dQZF1DWTzZ4lK0sd5v","37i9dQZF1DX9RGlg95cyIv","37i9dQZF1DZ06evO2AiJvF","37i9dQZF1DX5L9gRaHajqj")
    #for x in list_heavymetal:
    #  new_list = analyze_playlist("spotify", x, "Heavy Metal") 
    #  spotify_df = pd.concat([spotify_df, new_list])

    #list_rocknroll = ("37i9dQZF1DX0LonJ1uN97e","37i9dQZF1DZ06evO2cIVLk","37i9dQZF1DX50yT1VeARvT","37i9dQZF1DZ06evO0LSu7o","37i9dQZF1DWWypndwwoaGW","37i9dQZF1DZ06evO01jagL","37i9dQZF1DZ06evO23huA8","37i9dQZF1DZ06evO2FgIak","37i9dQZF1DZ06evO3uHZ0M","37i9dQZF1DZ06evO1twRXe","37i9dQZF1DZ06evO1eY6wU","37i9dQZF1DZ06evO280DHm","37i9dQZF1DZ06evO36IjyD","37i9dQZF1DZ06evO0mZ0as","37i9dQZF1DZ06evO4aKwBf","37i9dQZF1DZ06evO2GMuXd")
    #for x in list_rocknroll:
    #  new_list = analyze_playlist("spotify", x, "Rock n Roll") 
    #  spotify_df = pd.concat([spotify_df, new_list])

    #list_punkrock = ("37i9dQZF1DZ06evO0OX351","37i9dQZF1DWWc7UGhP4Mpw","37i9dQZF1DWT1alfusUUiJ","37i9dQZF1DWVPJN9etkrEa","37i9dQZF1DX4EKARgqYPFZ","37i9dQZF1DZ06evO2QPsK1","37i9dQZF1DZ06evO2CrP3O","37i9dQZF1DZ06evO2CLCA8","37i9dQZF1DZ06evO2xY6H0","37i9dQZF1DZ06evO0ECSIM","37i9dQZF1DZ06evO3p6o79","37i9dQZF1DZ06evO2hmiRq","37i9dQZF1DZ06evO45NDU6","37i9dQZF1DZ06evO1vVlNV","37i9dQZF1DZ06evO358rmq","37i9dQZF1DZ06evO0HTmQY","37i9dQZF1DZ06evO0l9Z4c","37i9dQZF1DZ06evO0rXOzg","37i9dQZF1DZ06evO1e88yJ","37i9dQZF1DZ06evO2Eh9pN","37i9dQZF1DZ06evO1vQMup")
    #for x in list_punkrock:
    #  new_list = analyze_playlist("spotify", x, "Punk Rock") 
    #  spotify_df = pd.concat([spotify_df, new_list])

    spotify_df.to_csv('D:\spotify.csv',sep='\t', index = False, header=True)
else:
    print('******************* Ejecucion deshabilitada *******************')

spotify_df.info() 

playlist_id: 37i9dQZF1DX6bnzK9KPvrz
<class 'pandas.core.frame.DataFrame'>
Int64Index: 65 entries, 0 to 64
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   genre               65 non-null     object 
 1   artist              65 non-null     object 
 2   album               65 non-null     object 
 3   track_name          65 non-null     object 
 4   track_id            65 non-null     object 
 5   danceability        65 non-null     float64
 6   energy              65 non-null     float64
 7   key                 65 non-null     object 
 8   loudness            65 non-null     float64
 9   mode                65 non-null     object 
 10  speechiness         65 non-null     float64
 11  instrumentalness    65 non-null     float64
 12  valence             65 non-null     float64
 13  tempo               65 non-null     float64
 14  time_signature      65 non-null     object 
 15  acousticness        65 

---

## <b> <a id="Genius">2. Utilizando Genius para descargar los Lyrics de cada Track </a> </b>

A partir del dataset descargado de Spotify con los atributos de cada canción, ahora vamos a utilizar la API de Genius para descargar los lyrics de cada Track

### Configuración Genius: lyricsgenius (Generar access token en Genius.com)

In [16]:
genius = lg.Genius("LGJLiPXVfqz2Ar3oBUZGbq5VWF7b60Ei8_Caxv3Z3Tbu4Aoyt68kYhWFrxxfz681")
genius.verbose = True # Turn off status messages
genius.remove_section_headers = True # Remove section headers (e.g. [Chorus]) from lyrics when searching
genius.skip_non_songs = True # Include hits thought to be non-songs (e.g. track lists)
genius.excluded_terms = ["(Remix)", "(Live at)", "(Remastered)", "(Remaster)"] # Exclude songs with these words in their tit
genius.replace_default_terms = True

In [17]:
df = pd.read_csv('D:\spotify.csv', sep="\t")
df.shape

(65, 19)

Vamos a limpiar los track_name ya que detectamos que hay muchos que tienen los siguientes patrones
" - Remastered" "- Live At" "Remix" etc. y esto provoca que la descarga de Genius no sea la esperada ya que no descarga el Lyric sino entrevistas/traducciones/etc (u otras cosas)

In [18]:
ejecutar = True ## '******************* Ejecucion deshabilitada *******************'
if ejecutar:
    title_pattern = "(?P<title>^[^-]+([^-]+))" ## buscamos un titulo (caracteres) hasta el primer '-' y eliminamos el resto. Por ejemplo si la cancion se llama "Take on Me - Remastered 2001" nos queda "Take on Me"
    title_regex =  re.compile(title_pattern)
    title_match = df.track_name.apply(lambda x: x if x is np.NaN else title_regex.search(x))
    title_match_mask = title_match.notnull()
    df["track_name_regex"] = df["track_name"]
    df.loc[title_match_mask, "track_name_regex"] = title_match[title_match_mask].apply(lambda x: x.group("title"))    
    #df.to_csv('D:\spotify_lyrics_regex.csv', sep='\t', index = False, header=True) ## guardamos la data con el track_name_regex que es el titulo limpio.
else:
    print('******************* Ejecucion deshabilitada *******************')

---

A partir de acá, con el dataset mas limpio, especificamente el track_name, vamos a descargar los Lyric de Genius.
Vamos a buscar para cada Track, el Lyric en Genius, hacemos un For por Artist ya que por timeouts necesitabamos guardar por lotes para no perder informacion.

##### <b> NO EJECUTAR CON TODOS EL LISTADO DE TRACKS YA QUE DEMORA MUCHAS HORAS. </b> 

In [28]:
#df = pd.read_csv('D:\spotify_track_names_clean.csv', sep="\t")
data = df
data['lyric_clean'] = 'None' ## Columna donde descargamos el Lyric de Genius

ejecutar = True ## '******************* Ejecucion deshabilitada *******************'
if ejecutar:
    
    ###reprocess si falla el proceso, setear que genero falta procesar y first=false y re-ejecutar
    #df_mask_reprocess = (df["genre"] == 'Punk Rock') | (df["genre"] == 'Jazz') 
    #data = df[df_mask_reprocess]
    first = bool(True) #save headers only first time
    ###

    for artist in data['artist'].unique(): ## por cada artista en data
        artist_mask = data['artist'] == artist
        data_masked = data[artist_mask] ## traemos los track_name del artista
        try:
            for index, row in data_masked.iterrows():
                if row['lyric_clean'] == 'None': ## sólo buscamos los Lyrics que tengamos en valor None (esta validacion es por si reejecutamos, continuar con los "pendientes: none")
                    song = genius.search_song(row['track_name_regex'], row['artist']) ## buscamos en Genius API el Lyric por traack_name_regex y artist
                    if song is None: ## si no existe..
                        print(row['track_name_regex'] + ' no existe *********')
                    else: ## si existe  guardamos el song.Lyric en data_masked
                        data_masked.loc[(data['track_name_regex'] == row['track_name_regex']) & (data['artist'] == row['artist']), 'lyric_clean'] = song.lyrics ##seteamos el Lyric para el track_name y artist que buscamos
            if first: ##si es la primera vez que ejecutamos, guardamos el header, sino solo appendeamos
                data_masked.to_csv('D:\spotify_lyrics.csv', sep='\t', index = False, header=True)
                first = bool(False)
            else:
                data_masked.to_csv('D:\spotify_lyrics.csv', mode='a', sep='\t', index = False, header=False)

        except Exception as e: ##si falla continuamos, queda en None, y en una proxima ejecución reintentamos
            print(str(e))
            continue
else:
    print('******************* Ejecucion deshabilitada *******************')


Searching for "Starboy" by The Weeknd...
Done.
Searching for "The Hills" by The Weeknd...
Done.
Searching for "Can't Feel My Face" by The Weeknd...
Done.
Searching for "I Feel It Coming" by The Weeknd...
Done.
Searching for "Save Your Tears" by The Weeknd...
Done.
Searching for "Earned It (Fifty Shades Of Grey) " by The Weeknd...
Done.
Searching for "House Of Balloons / Glass Table Girls" by The Weeknd...
Done.
Searching for "Blinding Lights" by The Weeknd...
Done.
Searching for "In Your Eyes" by The Weeknd...
Done.
Searching for "Pray For Me (with Kendrick Lamar)" by The Weeknd...
Done.
Searching for "Heartless" by The Weeknd...
Done.
Searching for "Often" by The Weeknd...
Done.
Searching for "Call Out My Name" by The Weeknd...
Done.
Searching for "Die For You" by The Weeknd...
Done.
Searching for "Acquainted" by The Weeknd...
Done.
Searching for "Wicked Games" by The Weeknd...
Done.
Searching for "The Morning" by The Weeknd...
Done.
Searching for "After Hours" by The Weeknd...
Done.


Aún con la limpieza del paso anterior, hay descargas que no son Lyrics, por lo que analizamos la cantidad de caracteres de cada uno, si excede los 5000, los marcamos como "ignore" para filtrar mas adelante

In [30]:
ejecutar = True ## '******************* Ejecucion deshabilitada *******************'
if ejecutar:
    df_last_filtered = pd.read_csv('D:\spotify_lyrics.csv', sep="\t") ### ver nombre
    for index, row in df_last_filtered.iterrows():
        if (len(row['lyric_clean'])) > 5000: ## si la cancion tiene mas de 5000 caracteres, lo ignoramos ya que no es normal y puede ser algo distinto a un Lyric, no lo eliminamos para analizarlo si es necesario.
            df_last_filtered.loc[(df_last_filtered['track_name_regex'] == row['track_name_regex']) & (df_last_filtered['artist'] == row['artist']), 'ignore'] =  1
        else:
            df_last_filtered.loc[(df_last_filtered['track_name_regex'] == row['track_name_regex']) & (df_last_filtered['artist'] == row['artist']), 'ignore'] =  0
        df_last_filtered.loc[(df_last_filtered['track_name_regex'] == row['track_name_regex']) & (df_last_filtered['artist'] == row['artist']), 'char_count'] =  len(row['lyric_clean'])

    df_last_filtered.rename(columns={"track_name_regex": "track_name_clean", "ignore": 'ignore_track'}, inplace = True) ## renombramos columnas y dropeamos lyric.
    df_last_filtered.to_csv('D:\spotify_lyrics.csv', sep='\t', index = False, header=True)
else:
    print('******************* Ejecucion deshabilitada *******************')
    

## <b> <a id="Lexicon">3. Yyyyy.. entonces que vamos a hacer con toda la info de Spotify y Genius? Vamos a implementar Lexicon </a> </b> 

Seteamos el archivo con diccionario de palabras clasificadas en emociones (Descargado de https://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm)

In [31]:
lexicon = EmoLex("lexicon_english.txt")

Tambien vamos a cargar el diccionnario en un dataframe para poder realizar una busqueda de palabras 

In [32]:
data_lexicon = pd.read_csv('lexicon_english.txt',  sep="\t", header=None)
data_lexicon.head(5)

,0,1,2
0,aback,anger,0
1,aback,anticipation,0
2,aback,disgust,0
3,aback,fear,0
4,aback,joy,0


Seteo Stopwords y agregado de nuevas en base a las detectadas en los Lyrics de Genius

In [33]:
stopwords = stopwords.words("english")
additional_stopwords = ["mr", "a-you", "ca", "whoa", "'ve", "got", "'s", "+'", "'m", "'s", "+", "'re", ".'", "?", "?'", ".", "...", "oh", "'", "n't", "!", "hoo", "ah", ")", "(", "{", "}", "''", "1", "2", "I", "[", "]", ",", ":", "*", "-", "_", ";", "``", "/", 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
stopwords.extend(additional_stopwords)

## <b> Vamos a contabilizar la cantidad de palabras de cada sentimiento que tienen nuestros Lyrics </b>
    
Para nuestra implementamos vamos a hacer dos cosas, analizar las palabras en sentimientos y tambien vamos a analizar las palabras de los sentimientos positivos "positives" y tristes "sadness"

Para saber cuantas palabras aparecen en el diccionario, primero creamos 2 array con las palabras calificadas como positives y otro como sadness.

Luego para saber cuantas palabras de cada tipo Tokenizamos los Lyric, y sumarizmos las apariciones de cada una en el diccionario

Armamos columnas nuevas para determinar cuantas palabras positives y sadness tiene cada canción

Esta ejecución crea una columna por cada palabra del diccionario. Luego con estas columnas contamos la cantidad de veces que esa palabra aparece en cada lyric.

##### <b> NO EJECUTAR CON TODOS EL LISTADO DE TRACKS YA QUE DEMORA MUCHAS HORAS. </b> 

In [34]:
%%time

data = pd.read_csv('D:/spotify_lyrics.csv', sep="\t")
data.shape

data_mask = (data["ignore_track"] == 0) & (data["lyric_clean"] != "None") #nos quedamos solo con los tracks que tienen Lyric_clean y no debemos ignorar
data_lexi = data[data_mask]

#data_lexi = data_lexi.sample(10)

data_lexi['tokens'] = pd.Series(dtype=object)

ejecutar = True ## '******************* Ejecucion deshabilitada *******************'
if ejecutar:

    data_lexicon_mask = (data_lexicon[1] == 'positive') & (data_lexicon[2] == 1) #buscamos palabras positivas dentro del data_lexicon y armamos una lista
    positive_words = pd.Series(data_lexicon[data_lexicon_mask][0])

    data_lexicon_mask = (data_lexicon[1] == 'sadness') & (data_lexicon[2] == 1) #buscamos palabras tristes dentro del data_lexicon y armamos una lista
    sad_words = pd.Series(data_lexicon[data_lexicon_mask][0])

    for index, row in data_lexi.iterrows(): ##vamos a interar nuestro dataset, donde tenemos todos los Lyric que buscamos en Sptoify y Genius.

        tokens = nltk.word_tokenize(row["lyric_clean"].lower()) ## Tokenizamos el Lyric
        tokens_wo_stopwords = [token for token in tokens if token not in stopwords] ## ignoramos las que sean stopwords
        data_lexi.at[index, 'tokens'] = tokens_wo_stopwords        
        
        for word in positive_words: ## comparamos las positive_words que definimos con los tokens y nos guardamos cada palabra en una columa con prefjo "l" love (positives)
            column_name = 'l_' + word + '_quantity' ## armamos el nombre de la columna nueva 'dinamicamente'
            quantity = sum(word in s for s in tokens) ## sumamos la cantidad de veces que aparece esa palabra en nuestro Lyric
            data_lexi.loc[index, column_name] = quantity ## guardamos el dato de la cantidad en la columna nueva que seria la palabra asociada a un sentimiento de positividad

        for word in sad_words: ## comparamos las sad_words que definimos con los tokens y nos guardamos cada palabra en una columa con prefjo "s" sad
            column_name = 's_' + word + '_quantity' ## armamos el nombre de la columna nueva 'dinamicamente'
            quantity = sum(word in s for s in tokens) ## sumamos la cantidad de veces que aparece esa palabra en nuestro Lyric   
            data_lexi.loc[index, column_name] = quantity ## guardamos el dato de la cantidad en la columna nueva que seria la palabra asociada a un sentimiento de tristeza
else:
    print('******************* Ejecucion deshabilitada *******************')

data_lexi.head(5) ##verificamos lo obtenido


<timed exec>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Andres\anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Andres\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

Wall time: 1min 4s


,genre,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,...,s_wound_quantity,s_wrangling_quantity,s_wreck_quantity,s_wrecked_quantity,s_wretch_quantity,s_wretched_quantity,s_wrinkled_quantity,s_wrongdoing_quantity,s_wrongful_quantity,s_wrongly_quantity
0,Pop,The Weeknd,Starboy,Starboy,7MXVkk9YMctZqd1Srtv4MB,0.679,0.587,7,-7.015,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Pop,The Weeknd,Beauty Behind The Madness,The Hills,7fBv7CLKzipRk6EC6TWHOB,0.585,0.564,0,-7.063,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Pop,The Weeknd,Beauty Behind The Madness,Can't Feel My Face,22VdIZQfgXJea34mQxlt81,0.705,0.769,9,-5.526,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Pop,The Weeknd,Starboy,I Feel It Coming,3dhjNA0jGA8vHBQ1VdD6vV,0.773,0.819,0,-5.946,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Pop,The Weeknd,After Hours,Save Your Tears,5QO79kh1waicV47BqGRL3g,0.680,0.826,0,-5.487,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## <b> Acá vamos a guardar los sentimientos clasificados por Lexicon </b>
Cada sentimiento se guardará en nuevas columnas, este dato representa una proporcion de la cantidad de veces que aparece dicho sentimiento sobre la cantidad de Tokens del Lyric

In [37]:
%%time
data_lexi

for index, _ in data_lexi.iterrows():
    try:
        to_lower = list(map(lambda x:x.lower(), data_lexi.loc[index].tokens))
        summary = lexicon.summarize_doc(to_lower)
        for key in summary.keys():
            data_lexi.at[index, key] = summary[key]
    except:
        continue
data_lexi

C:\Users\Andres\anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Andres\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Wall time: 210 ms


,genre,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,...,positive,joy,anticipation,trust,sadness,negative,anger,fear,disgust,surprise
0,Pop,The Weeknd,Starboy,Starboy,7MXVkk9YMctZqd1Srtv4MB,0.679,0.587,7,-7.015,1,...,0.127119,0.088983,0.055085,0.063559,0.059322,0.084746,0.033898,0.067797,0.008475,0.021186
1,Pop,The Weeknd,Beauty Behind The Madness,The Hills,7fBv7CLKzipRk6EC6TWHOB,0.585,0.564,0,-7.063,0,...,0.114130,0.048913,0.032609,0.081522,0.005435,0.005435,0.000000,0.027174,0.000000,0.000000
2,Pop,The Weeknd,Beauty Behind The Madness,Can't Feel My Face,22VdIZQfgXJea34mQxlt81,0.705,0.769,9,-5.526,0,...,0.255474,0.255474,0.065693,0.000000,0.065693,0.080292,0.021898,0.065693,0.021898,0.021898
3,Pop,The Weeknd,Starboy,I Feel It Coming,3dhjNA0jGA8vHBQ1VdD6vV,0.773,0.819,0,-5.946,0,...,0.031873,0.027888,0.250996,0.003984,0.003984,0.011952,0.007968,0.007968,0.007968,0.000000
4,Pop,The Weeknd,After Hours,Save Your Tears,5QO79kh1waicV47BqGRL3g,0.680,0.826,0,-5.487,1,...,0.099291,0.092199,0.021277,0.085106,0.078014,0.085106,0.007092,0.035461,0.000000,0.007092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Pop,NAV,NAV,Some Way,7zxRMhXxJMQCeDDg0rKAVo,0.744,0.715,0,-6.103,1,...,0.045833,0.008333,0.025000,0.020833,0.029167,0.087500,0.029167,0.029167,0.037500,0.012500
60,Pop,Gesaffelstein,Lost in the Fire,Lost in the Fire,7wFybC8jBH3zE139OpCtpG,0.663,0.675,2,-12.159,1,...,0.047393,0.037915,0.014218,0.033175,0.180095,0.189573,0.165877,0.189573,0.165877,0.165877
61,Pop,Belly,Up For Days,Might Not,5e9aFSdnkQUrm5y8Dhw0qN,0.792,0.563,1,-4.836,1,...,0.029520,0.018450,0.040590,0.014760,0.011070,0.055351,0.033210,0.014760,0.018450,0.007380
62,Pop,Drake,Take Care (Deluxe),Crew Love,0TiC3GtlMCskf2hIUIBcDV,0.418,0.546,0,-9.057,0,...,0.131868,0.109890,0.038462,0.214286,0.021978,0.038462,0.027473,0.010989,0.021978,0.010989


Ahora vamos a ver que palabras tienen mas presencia en los Lyrics de nuestro dataset, para eso vamos a ver la media de cada palabra

In [38]:
data_describe = data_lexi.describe() 

Dropeamos las columnas no asociadas a palabras

In [39]:
data_describe.drop(columns=['key', 'speechiness', 'mode', 'instrumentalness', 'valence', 'danceability', 'energy', 'loudness', 'tempo', 'duration_s', 'album_release_year', 'char_count', 'time_signature', 'acousticness', 'track_popularity', 'ignore_track'], inplace=True)
data_describe

,l_abba_quantity,l_ability_quantity,l_abovementioned_quantity,l_absolute_quantity,l_absolution_quantity,l_absorbed_quantity,l_abundance_quantity,l_abundant_quantity,l_academic_quantity,l_academy_quantity,...,positive,joy,anticipation,trust,sadness,negative,anger,fear,disgust,surprise
count,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,...,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.095290,0.070433,0.052589,0.043209,0.052280,0.078663,0.031483,0.046269,0.027766,0.024660
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.062553,0.051203,0.048313,0.041698,0.051398,0.056927,0.034959,0.048348,0.033793,0.031938
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001342,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.050253,0.031800,0.025321,0.018807,0.019905,0.048335,0.010153,0.014679,0.008511,0.005497
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.077626,0.055556,0.038462,0.033175,0.036496,0.064516,0.022727,0.035156,0.019868,0.013889
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.124541,0.094079,0.065105,0.048132,0.067438,0.091974,0.038312,0.058039,0.032847,0.030598
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.333333,0.255474,0.250996,0.214286,0.236842,0.271930,0.174888,0.254386,0.179372,0.165877


## <b> Ahora que tengo 2312 columnas asociadas a palabras positive y 1191 sadness quiero saber cuales son las 40 mas presentes en los tracks </b> 

Para realizar una busqueda mas facil (para nosotros), tomamos la media de cada palabra, lo guardamos en un txt y lo leemos como un dataframe, 
para luego determinar cuales son los 40 palabras más frecuentes asociadas a los sentimientos positivos y negativos 

Este paso lo hacemos de esta forma porque nos resulto más facil para poder obtener las palabras mas repetidas, nos guardamos en un TXT las medias de cada palabra encontrada en el diccionario en un formato "columnas" y luego lo leemos como un dataframe. 


In [40]:

mean_lexi = []
for column in data_describe: ## recorremos cada columna del data_describe y lo appendeamos a una lista
    mean_lexi.append(data_describe[column].name + ' \t ' + str(data_describe[column]["mean"]))
dfl = pd.DataFrame(mean_lexi) ## convertimos la lista como un dataframe

dfl.to_csv('D:\mean_lexi.txt', index=False, header=False) ## guardamos el dataframe
mean_lexi = pd.read_csv('D:\mean_lexi.txt', sep="\t", header=None) ## leemos el dataframe 


Con el archivo que generamos en el paso anterior (un data frame de 2 columnas: palabra y media) vamos a elegir las 40 palabras con la media mas alta
Vamos a dropear todas las columnas que no nos interesan y con esto falta un paso para llegar al dataset final

In [42]:
mean_lexi_40 = mean_lexi.sort_values(by=1, ascending=False).head(40)
df_principal_words = pd.DataFrame(mean_lexi_40)
df_principal_words.reset_index(drop=True, inplace=True)

ejecutar = True ## '******************* Ejecucion deshabilitada *******************'
if ejecutar:

    list_columns_include = []
    list_columns_include.append('speechiness')
    list_columns_include.append('mode')
    list_columns_include.append('genre')
    list_columns_include.append('instrumentalness')
    list_columns_include.append('valence')
    list_columns_include.append('danceability')
    list_columns_include.append('energy')
    list_columns_include.append('loudness')
    list_columns_include.append('tempo')
    list_columns_include.append('album_release_year')
    list_columns_include.append('char_count')
    list_columns_include.append('time_signature')
    list_columns_include.append('acousticness')
    list_columns_include.append('artist')
    list_columns_include.append('album')
    list_columns_include.append('track_name_clean')
    list_columns_include.append('ignore_track')
    list_columns_include.append('char_count')
    list_columns_include.append('anger') 
    list_columns_include.append('anticipation')
    list_columns_include.append('disgust') 
    list_columns_include.append('fear') 
    list_columns_include.append('joy') 
    list_columns_include.append('negative') 
    list_columns_include.append('positive')
    list_columns_include.append('sadness') 
    list_columns_include.append('surprise')
    list_columns_include.append('trust') 
    list_columns_include.append('tokens')

    for item in df_principal_words[0]:
        #print(item)
        list_columns_include.append(str.rstrip(item))

    data_mask = (data_lexi["ignore_track"] == 0) & (data_lexi["lyric_clean"] != "None") #nos quedamos solo con los tracks que tienen Lyric_clean y no debemos ignorar
    data_words = data_lexi[data_mask]
    data_words.drop(data_words.columns.difference(list_columns_include), 1, inplace=True)
    data_words.reset_index(drop=True, inplace=True)
    #data_words.head(3)

    data_words.to_csv('D:\spotify_words_filtered.csv', index=False)
else: 
    print('******************* Ejecucion deshabilitada *******************')
#list_columns_include


In [43]:
data_words.columns

Index(['genre', 'artist', 'album', 'danceability', 'energy', 'loudness',
       'mode', 'speechiness', 'instrumentalness', 'valence', 'tempo',
       'time_signature', 'acousticness', 'album_release_year',
       'track_name_clean', 'ignore_track', 'char_count', 'tokens',
       'l_ace_quantity', 'l_aga_quantity', 'l_aid_quantity', 'l_ally_quantity',
       'l_art_quantity', 'l_ay_quantity', 'l_baby_quantity', 'l_don_quantity',
       'l_eat_quantity', 'l_feat_quantity', 'l_friend_quantity',
       'l_fun_quantity', 'l_gain_quantity', 'l_good_quantity',
       'l_king_quantity', 'l_land_quantity', 'l_love_quantity',
       'l_money_quantity', 'l_pad_quantity', 'l_real_quantity',
       'l_rock_quantity', 'l_star_quantity', 'l_talk_quantity',
       'l_top_quantity', 'l_wit_quantity', 's_art_quantity',
       's_bitch_quantity', 's_dark_quantity', 's_die_quantity',
       's_heartless_quantity', 's_hurt_quantity', 's_ill_quantity',
       's_lie_quantity', 's_lone_quantity', 's_lose_qua

# <b> Tambien queremos saber la cantidad de "sentimientos" aparece en cada Lyric</b> 

entonces vamos a crear columnas nuevas para cada sentimiento y luego vamos a tokenizar nuevamente y utilizar el metodo summarize_doc


In [44]:

data_words['anger_quantity'] = 0
data_words['anticipation_quantity'] = 0
data_words['disgust_quantity'] = 0
data_words['fear_quantity'] = 0
data_words['joy_quantity'] = 0
data_words['negative_quantity'] = 0
data_words['positive_quantity'] = 0
data_words['sadness_quantity'] = 0
data_words['surprise_quantity'] = 0
data_words['trust_quantity'] = 0

data_words


,genre,artist,album,danceability,energy,loudness,mode,speechiness,instrumentalness,valence,...,anger_quantity,anticipation_quantity,disgust_quantity,fear_quantity,joy_quantity,negative_quantity,positive_quantity,sadness_quantity,surprise_quantity,trust_quantity
0,Pop,The Weeknd,Starboy,0.679,0.587,-7.015,1,0.2760,0.000006,0.486,...,0,0,0,0,0,0,0,0,0,0
1,Pop,The Weeknd,Beauty Behind The Madness,0.585,0.564,-7.063,0,0.0515,0.000000,0.137,...,0,0,0,0,0,0,0,0,0,0
2,Pop,The Weeknd,Beauty Behind The Madness,0.705,0.769,-5.526,0,0.0425,0.000000,0.583,...,0,0,0,0,0,0,0,0,0,0
3,Pop,The Weeknd,Starboy,0.773,0.819,-5.946,0,0.1180,0.000000,0.585,...,0,0,0,0,0,0,0,0,0,0
4,Pop,The Weeknd,After Hours,0.680,0.826,-5.487,1,0.0309,0.000012,0.644,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Pop,NAV,NAV,0.744,0.715,-6.103,1,0.3510,0.000000,0.340,...,0,0,0,0,0,0,0,0,0,0
59,Pop,Gesaffelstein,Lost in the Fire,0.663,0.675,-12.159,1,0.0359,0.001330,0.176,...,0,0,0,0,0,0,0,0,0,0
60,Pop,Belly,Up For Days,0.792,0.563,-4.836,1,0.0787,0.000000,0.321,...,0,0,0,0,0,0,0,0,0,0
61,Pop,Drake,Take Care (Deluxe),0.418,0.546,-9.057,0,0.2380,0.000000,0.294,...,0,0,0,0,0,0,0,0,0,0


In [45]:
%%time

data_process = data_words

for index, _ in data_process.iterrows():
    try:
        to_lower = list(map(lambda x:x.lower(), data_process.loc[index].tokens))
        summary = lexicon.summarize_doc(to_lower)
        for key in summary.keys():
            data_process.at[index, key] = summary[key]
    except:
        continue
        
## probamos que este OK        
data_mask = (data_process["positive"] > 0) | (data_process["fear"] > 0)
data_process[data_mask]["positive"]
#data_process


Wall time: 50.9 ms


0     0.127119
1     0.114130
2     0.255474
3     0.031873
4     0.099291
        ...   
58    0.045833
59    0.047393
60    0.029520
61    0.131868
62    0.001342
Name: positive, Length: 63, dtype: float64

### Contamos la cantidad de veces que el sentimiento anger/fear/etc aparece en cada lyric, y guardamos el dataset final: music_sentiments.csv

In [47]:
%%time

ejecutar = True ## '******************* Ejecucion deshabilitada *******************'
if ejecutar:    
    for index, cancion in data_process.iterrows():
        annotate = lexicon.annotate_doc(cancion.tokens)
        lista = []
        for items in annotate:
            if items != set():
                for emotion in items:
                    lista.append(emotion)
        data_process.loc[index, 'anger_quantity'] = Counter(lista)["anger"]
        data_process.loc[index, 'anticipation_quantity'] = Counter(lista)["anticipation"]
        data_process.loc[index, 'disgust_quantity'] = Counter(lista)["disgust"]
        data_process.loc[index, 'fear_quantity'] = Counter(lista)["fear"]
        data_process.loc[index, 'joy_quantity'] = Counter(lista)["joy"]
        data_process.loc[index, 'negative_quantity'] = Counter(lista)["negative"]
        data_process.loc[index, 'positive_quantity'] = Counter(lista)["positive"]
        data_process.loc[index, 'sadness_quantity'] = Counter(lista)["sadness"]
        data_process.loc[index, 'surprise_quantity'] = Counter(lista)["surprise"]
        data_process.loc[index, 'trust_quantity'] = Counter(lista)["trust"]
        
    data_process.to_csv('D:\music_sentiments.csv', sep="\t", index=None)
else:
    print('******************* Ejecucion deshabilitada *******************')


Wall time: 174 ms


# <b> Terminado! Tenemos nuestro dataset music_sentiments </b> 

In [50]:

data = pd.read_csv('D:/music_sentiments.csv', sep="\t")
data_mask = data["artist"] == 'The Weeknd'
data[data_mask]


,genre,artist,album,danceability,energy,loudness,mode,speechiness,instrumentalness,valence,...,anger_quantity,anticipation_quantity,disgust_quantity,fear_quantity,joy_quantity,negative_quantity,positive_quantity,sadness_quantity,surprise_quantity,trust_quantity
0,Pop,The Weeknd,Starboy,0.679,0.587,-7.015,1,0.2760,0.000006,0.4860,...,8,13,2,16,21,20,30,14,5,15
1,Pop,The Weeknd,Beauty Behind The Madness,0.585,0.564,-7.063,0,0.0515,0.000000,0.1370,...,0,6,0,5,9,1,21,1,0,15
2,Pop,The Weeknd,Beauty Behind The Madness,0.705,0.769,-5.526,0,0.0425,0.000000,0.5830,...,3,9,3,9,35,11,35,9,3,0
3,Pop,The Weeknd,Starboy,0.773,0.819,-5.946,0,0.1180,0.000000,0.5850,...,2,63,2,2,7,3,8,1,0,1
4,Pop,The Weeknd,After Hours,0.680,0.826,-5.487,1,0.0309,0.000012,0.6440,...,1,3,0,5,13,12,14,11,1,12
5,Pop,The Weeknd,Trilogy,0.650,0.711,-5.417,1,0.0377,0.009430,0.3190,...,7,43,10,9,44,19,49,10,5,18
6,Pop,The Weeknd,After Hours,0.514,0.730,-5.934,1,0.0598,0.000095,0.3340,...,2,4,1,1,4,8,5,1,1,4
7,Pop,The Weeknd,After Hours,0.667,0.719,-5.371,0,0.0346,0.000081,0.7170,...,4,1,4,8,4,17,5,11,3,3
8,Pop,The Weeknd,Pray For Me (with Kendrick Lamar),0.728,0.668,-5.020,1,0.1260,0.000015,0.1620,...,9,12,5,12,9,15,21,9,6,6
9,Pop,The Weeknd,After Hours,0.537,0.746,-5.507,0,0.1500,0.000001,0.2520,...,10,17,5,8,11,37,14,35,9,7
